In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from src import Spotify
from src.utils import KMeans, DB

# My Data Setting

In [2]:
my_sp = Spotify()
my_sp.search()

계속하시려면 아무거나, 종료는 exit를 입력해주세요.exit


In [3]:
db = DB()
save_mb = db.save_mailbox(my_sp.sel_tracks)

# Recommender Init
- 실제로는 DB에 들어가서 Recommender가 빼는 형태라서, my_sp를 Recommender에 넣어서 빼는 작업이 필요

In [34]:
import pandas as pd

In [47]:
save_mb.inserted_id

ObjectId('6265f7aafe9fef932765c60b')

## init_setting
- Spotify 로 부터 추천 음악들을 긁어오는 과정

In [48]:
mail_box_id = str(save_mb.inserted_id)
mail_box = db.get_mailbox(mail_box_id)

cols = ['trackId', 'trackName', 'artistIds', 'artistNames', 'image']

_sel_tracks = mail_box['tracks']
sel_tracks = pd.DataFrame(_sel_tracks)[cols]

sp = Spotify(sel_tracks)
sp.get_genres()
sp.get_features()
sp.get_recommend()

reco_sp = Spotify(sp.reco_tracks)
reco_sp.get_features()

my_tracks = sp.sel_tracks
my_features = sp.features

reco_tracks = reco_sp.sel_tracks
reco_features = reco_sp.features

## Data Preprocessing

In [54]:
features = pd.concat([my_features, reco_features])

norm_features = features.values[:, 1:]
norm_features = (norm_features - norm_features.min(axis=0)) / \
                (norm_features.max(axis=0) - norm_features.min(axis=0))

norm_features

array([[0.18497109826589597, 0.7459936770958248, 0.7272727272727273, ...,
        0.09711322869955157, 0.38195732838589985, 0.2862043711213043],
       [0.541907514450867, 0.7732475744031397, 0.45454545454545453, ...,
        0.06852578475336323, 0.9628942486085345, 0.6939675442315846],
       [0.758670520231214, 0.5955521639594462, 0.0, ...,
        0.0709080717488789, 0.7541743970315399, 0.33915892533631414],
       ...,
       [0.6936416184971098, 0.5399542134525237, 0.18181818181818182, ...,
        0.11294843049327352, 0.3262987012987013, 0.49302702077631727],
       [0.6546242774566474, 0.7350921181728988, 0.7272727272727273, ...,
        0.04344170403587443, 0.7959183673469389, 0.5543769032108854],
       [0.5939306358381504, 0.9400414259239072, 0.5454545454545454, ...,
        0.24887892376681614, 0.5987940630797773, 0.6934587364607023]],
      dtype=object)